In [5]:
from IPython.display import Latex

#### robot, encoder, hdt. hdt to torque formaula, motor current torque estimate.

## 3.1 Kinematics

This section introduces some basic concepts and algorithmic implementations about the kinematics of a robot
used in the rest of the thesis. The conversions between coordinate frames and the chosen convention are covered in the first sub-section. The forward kinematics and the geometric Jacobian computation process are then briefly explained, along with its implementation

### 3.1.1 Homogeneous Transformations


Homogeneous transformations are a popular and effective method for representing the relationships between coordinate frames, widely used in robotics, computer graphics, and computer vision <cite id="fq99f"><a href="#zotero%7C17740253%2FLY2FP5ZV">(Ghosal, 2006)</a></cite>. To understand how these transformations are derived, consider two reference frames, $\text{RF}_{i-1}$ and $\text{RF}_i$, with origins $O_{i-1}$ and $O_i$, respectively. Suppose there is a point $P$, represented by the vector ${}^{i}\mathbf{x}_i \in \mathbb{R}^3$ in the frame $\text{RF}_i$. The goal is to express this point in the coordinate frame $\text{RF}_{i-1}$. Through basic geometric considerations, the point $p$ in frame $\text{RF}_{i-1}$, represented by the vector ${}^{i-1}\mathbf{x}_i$, is obtained as follows:
\begin{equation}
{}^{i-1}\mathbf{x}_i = {}^{i-1}\mathbf{R}_i \, {}^{i}\mathbf{x}_i + {}^{i-1}\mathbf{p}_i,
\label{eq:3.1} \tag{3.1}
\end{equation}

where ${}^{i-1}\mathbf{R}_{i} \in \text{SO}(3)$ is a rotation matrix describing the rotation from $\text{RF}_i$ to $\text{RF}_{i-1}$, and ${}^{i-1}\mathbf{p}_{i}\in \mathbb{R}^3$ is the translation vector between the two frames. The set $\text{SO}(3)$ consists of all orthogonal matrices with a determinant of +1, representing pure rotation matrices.

To use homogeneous transformations,  a fourth component with a value of one is added to the vector $\mathbf{x}$, transforming it into a homogeneous vector $\tilde{\mathbf{x}}$:

\begin{equation}
\tilde{\mathbf{x}} = \begin{bmatrix} \mathbf{x} \\ 1 \end{bmatrix}.
\label{eq:3.2} \tag{3.2}
\end{equation}

This enables the transformation to be written in a compact form:
\begin{equation}
{}^{i-1}\tilde{\mathbf{x}}_i =
\underbrace{
\begin{pmatrix}
{}^{i-1}\mathbf{R}_i & {}^{i-1}\mathbf{p}_i \\
\mathbf{0}^\mathrm{T} & 1
\end{pmatrix}}_{{} = {}^{i-1}\mathbf{T}_i}
{}^{i}\tilde{\mathbf{x}}_i,
\label{eq:3.3} \tag{3.3}
\end{equation}
The matrix ${}^{i-1}\mathbf{T}_i$, is known as the homogeneous transformation matrix. This matrix effectively contains all the necessary information to convert a point from one reference frame to another, integrating both rotation and translation in a single operation. 

### 3.1.2 Denavit-Hartenberg Convention


Although reference frames attached to the links can be chosen arbitrarily, it is practical to select them systematically. A widely used convention for describing the position and orientation of successive link frames relative to each other is the Denavit-Hartenberg (DH) convention <cite id="gx4bn"><a href="#zotero%7C17740253%2FLY2FP5ZV">(Ghosal, 2006)</a></cite><cite id="10z4u"><a href="#zotero%7C17740253%2F3J5DS9LQ">(Denavit &#38; Hartenberg, 2021)</a></cite>


#### define the parmaetrs 

The homogeneous transformation between frame $i$ and $i-1$ is obtained by multiplying the rotation $\mathbf{T}_{R_z}$ around the $\hat{\mathbf{z}}_{i-1}$-axis, the translation $\mathbf{T}_{T_z}$ along the $\hat{\mathbf{z}}_{i-1}$-axis, the translation $\mathbf{T}_{T_x}$ along the $\hat{\mathbf{x}}_i$-axis, and the rotation $\mathbf{T}_{R_x}$ around the $\hat{\mathbf{x}}_i$-axis, as follows:

\begin{equation}
{}^{i-1}\mathbf{T}_i(\theta_i, d_i, a_i, \alpha_i) = \mathbf{T}_{R_z}(\theta_i) \mathbf{T}_{T_z}(d_i) \mathbf{T}_{T_x}(a_i) \mathbf{T}_{R_x}(\alpha_i)
\label{eq:3.4} \tag{3.4}
\end{equation}

This sequence of transformations can be represented by the following matrix multiplications:

$$
\mathbf{T}_{R_z}(\theta_i) = \begin{bmatrix}
c_{\theta_i} & -s_{\theta_i} & 0 & 0 \\
s_{\theta_i} & c_{\theta_i} & 0 & 0 \\
0 & 0 & 1 & 0 \\
0 & 0 & 0 & 1
\end{bmatrix}
$$

$$
\mathbf{T}_{T_z}(d_i) = \begin{bmatrix}
1 & 0 & 0 & 0 \\
0 & 1 & 0 & 0 \\
0 & 0 & 1 & d_i \\
0 & 0 & 0 & 1
\end{bmatrix}
$$

$$
\mathbf{T}_{T_x}(a_i) = \begin{bmatrix}
1 & 0 & 0 & a_i \\
0 & 1 & 0 & 0 \\
0 & 0 & 1 & 0 \\
0 & 0 & 0 & 1
\end{bmatrix}
$$

$$
\mathbf{T}_{R_x}(\alpha_i) = \begin{bmatrix}
1 & 0 & 0 & 0 \\
0 & c_{\alpha_i} & -s_{\alpha_i} & 0 \\
0 & s_{\alpha_i} & c_{\alpha_i} & 0 \\
0 & 0 & 0 & 1
\end{bmatrix}
$$

Combining these, the homogeneous transformation matrix is:

\begin{equation}
{}^{i-1}\mathbf{T}_i(\theta_i, d_i, a_i, \alpha_i) = 
\begin{bmatrix}
c_{\theta_i} & -s_{\theta_i} c_{\alpha_i} & s_{\theta_i} s_{\alpha_i} & a_i c_{\theta_i} \\
s_{\theta_i} & c_{\theta_i} c_{\alpha_i} & -c_{\theta_i} s_{\alpha_i} & a_i s_{\theta_i} \\
0 & s_{\alpha_i} & c_{\alpha_i} & d_i \\
0 & 0 & 0 & 1
\end{bmatrix}
\label{eq:3.5} \tag{3.5}
\end{equation}

In this expression, $c_{\theta_i}$ and $s_{\theta_i}$ represent $\cos(\theta_i)$ and $\sin(\theta_i)$, respectively. Notably, the homogeneous transformation ${}^{i-1}\mathbf{T}_i(\theta_i, d_i, a_i, \alpha_i)$ depends on a single variable once the reference frame is established and the other parameters are fixed. This variable is determined by the type of joint: $d_i$ for prismatic joints and $\theta_i$ for revolute joints. A more elaborate theory can be found in <cite id="zhen5"><a href="#zotero%7C17740253%2FLY2FP5ZV">(Ghosal, 2006)</a></cite>.

In [14]:
def dh_transform(a, alpha, d, theta):
    """
    Compute Denavit-Hartenberg transformation matrix.
    """
    return np.array([
        [np.cos(theta), -np.sin(theta) * np.cos(alpha),  np.sin(theta) * np.sin(alpha), a * np.cos(theta)],
        [np.sin(theta),  np.cos(theta) * np.cos(alpha), -np.cos(theta) * np.sin(alpha), a * np.sin(theta)],
        [0,             np.sin(alpha),                 np.cos(alpha),                 d],
        [0,             0,                             0,                             1]
    ])

### 3.1.3 Forward Kinematics

The kinematic representation of a robot manipulator consists of multiple links connected by joints. The links are generally rigid bodies that form the robot’s structure, while the joints are the movable components that connect these links. Joints can be either active, with an actuator, or passive, without an actuator. Additionally, joints are commonly foound in serial robots are usually prismatic or revolute. Each type of such joint adds one degree of freedom (dof) to the system: the joint position $q_i$ for prismatic joints or the joint angle $q_i$ for revolute joints. The dof of an active joint can be controlled individually by a microcontroller, which relies on feedback from an encoder measuring the joint's position. The entire kinematic structure is typically referred to as a chain, which can be of two types:

* Open chain: This configuration has only one path in the structure that connects the two extremes, commonly referred to as the base and the end-effector.
* Closed chain: This configuration includes loops within the kinematic structure.


In this thesis, only active revolute joints arranged in an open chain are considered.

A common problem in robotics is determining the Cartesian position $\mathbf{x} \in \mathbb{R}^3$ of a specific point on a link given the joint angles $\mathbf{q} \in \mathbb{R}^n$. This problem is known as forward kinematics, which can be efficiently solved using homogeneous transformations. For instance, the position of the origin of the end-effector’s reference frame $\text{RF}_e$ in the base frame $\text{RF}_b$ can be determined using the following equation:

$$
{}^b\mathbf{T}_e(\mathbf{q}) = {}^b\mathbf{T}_0 \, {}^0\mathbf{T}_1(q_1) \cdots {}^{n-1}\mathbf{T}_e(q_n) \, {}^n\mathbf{T}_e,
$$

where ${}^b\mathbf{T}_0$ and ${}^n\mathbf{T}_e$ are two constant homogeneous transformations needed to convert a point from frame 0 to the base frame and from the end-effector frame to frame $n$, respectively. For a more detailed description, refer to the relevant section in
 <cite id="wc30q"><a href="#zotero%7C17740253%2FLY2FP5ZV">(Ghosal, 2006)</a></cite>.

### 3.1.4 Geometric Jacobian

The differential kinematic gives the relationship between the joint velocities q̇ and
the task space instantaneous velocities [ ṗ T , ω T ] T . where the vector ṗ = [ ẋ, ẏ, ż] T
describes the Cartesian linear velocities, and ω = [ω x , ω y , ω z ] T are the angular
velocities in the base frame. The matrix that maps these two quantities is called
geometric Jacobian J (q) ∈ R 6×n
$$
\begin{bmatrix}
\dot{\mathbf{p}} \\
\boldsymbol{\omega}
\end{bmatrix}
= \mathbf{J}(\mathbf{q}) \dot{\mathbf{q}}.
$$
The reason why it is called geometric Jacobian is that the derivation is done in a
geometric way and not directly via a time differentiation of the forward kinematics.
This last Jacobian is called analytic Jacobian. Throughout this thesis, Jacobians are
always meant to be the geometric Jacobians.
It is helpful for the derivation to partition the Jacobian into 3 × 1 sub-vectors called
j p,i and j o,i , where j p,i describes the contribution of joint i to the linear velocity
and j o,i (q) describes the contribution of joint i to the angular velocity. With the
partitioned Jacobian equation (2.6) is re-written as
$$
\begin{pmatrix}
\dot{\mathbf{p}} \\
\boldsymbol{\omega}
\end{pmatrix}
= \begin{pmatrix}
j_{p,1} & \cdots & j_{p,n} \\
j_{o,1} & \cdots & j_{o,n}
\end{pmatrix}
\dot{\mathbf{q}}.
$$
In the following, only revolute joints are considered for the Jacobian computation.
An exaustive explanation can be found in [18]. It is convenient to proceed separately
for the computation of the linear and angular velocity.
#### Linear Velocity
The linear velocity of the desired frame RF e due to the angular velocity of a
revolute joint i can be calculated as follows
$$
J_{p,i} \dot{q}_i = \boldsymbol{\omega}_{i-1,i} \times \mathbf{r}_{i-1,e} \\
= \dot{q}_i \mathbf{z}_{i-1} \times (\mathbf{p}_{0,e} - \mathbf{p}_{0,i-1}).
$$
The vector z i−1 describes the rotation axis of joint i, the vectors p 0,e and p 0,i−1 are
the position of frame RF e and i − 1 relative to the base frame and represented in frame i. The joint velocity of revolute joint i is denoted by q̇ i and
$$
\mathbf{J}_{p,i} = \mathbf{z}_{i-1} \times (\mathbf{p}_{0,s} - \mathbf{p}_{0,i-1}),
$$
holds.
#### Angular Velocity
The expression to calculate the relative angular velocity between two consecutive elements in the robot chain is given by
\begin{equation}
\boldsymbol{\omega}_{i-1,i} = \dot{q}_i \mathbf{z}_{i-1}
\end{equation}
where $\boldsymbol{\omega}_{i-1,i}$ describes the relative angular velocity from frame $i-1$ to frame $i$. For a revolute joint, the Jacobian that relates the angular velocity in the joint space to the angular velocity in the Cartesian space is given by
\begin{equation}
\mathbf{J}_{o,i} \dot{q}_i = \dot{q}_i \mathbf{z}_{i-1}
\end{equation}
and then
\begin{equation}
\mathbf{J}_{o,i} = \mathbf{z}_{i-1}
\end{equation}
where $\mathbf{z}_{i-1}$ is the third column of ${}^0\mathbf{R}_{i-1}$, hence the rotation axis of joint $i$, that is computed by
\begin{equation}
\mathbf{z}_{i-1} = {}^0\mathbf{R}_1(q_1) \cdots {}^{i-2}\mathbf{R}_{i-1}(q_{i-1}) \mathbf{z}_0.
\end{equation}
Here, $\mathbf{z}_0 = [0, 0, 1]^\mathrm{T}$ allows the selection of the third column.



In [19]:
def compute_geometric_jacobian(joint_angles,dh_params = 0):
    """
    Compute the geometric Jacobian for a robot given DH parameters and joint angles.

    :param dh_params: List of DH parameters [(a, alpha, d, theta), ...]
    :param joint_angles: List of joint angles [theta1, theta2, ..., thetaN]
    :return: Geometric Jacobian matrix (6 x N)
    """
    num_joints = len(joint_angles)
    T = np.eye(4)
    transformations = [T.copy()]
    
    # Compute forward kinematics to get transformations
    for i, (a, alpha, d, theta) in enumerate(dh_params):
        theta += joint_angles[i]  # Update theta with joint angle
        T_i = dh_transform(a, alpha, d, theta)
        T = np.dot(T, T_i)
        transformations.append(T.copy())
    
    # Initialize Jacobian matrix
    jacobian = np.zeros((6, num_joints))
    
    # End-effector position
    T_0_n = transformations[-1]
    o_n = T_0_n[:3, 3]

    # Compute each column of the Jacobian
    for i in range(num_joints):
        T_0_i = transformations[i]
        o_i = T_0_i[:3, 3]
        z_i = T_0_i[:3, 2]

        jacobian[:3, i] = np.cross(z_i, (o_n - o_i))  # Linear part
        jacobian[3:, i] = z_i  # Angular part

    return jacobian

## 3.2 LARA 8 Kinematic Description


![convert to latex](Resources/Lara8.png "Lara8")


The robot used for the experiments in this work is LARA 8, a collaborative robotic
arm from the company NEURA Robotics GmbH. It is a 6-axis serial kinematics
that has six independently actuated revolute joints and seven links. Figure[] shows the DH
frames, the DH parameters and the actuated joints of LARA 8. The DH-parameters
are given in Table [cite]

| frame | theta (rad) | d (m) | a (m) | alpha (rad) |
|---|---|---|---|---|
| K1 | 0 | 0.26 | 0 | $\frac{\pi}{2}$ |
| K2 | $\frac{\pi}{2}$ | 0 | 0.6 | $\pi$ |
| K3 | $\frac{\pi}{2}$ | 0 | 0 | $\frac{\pi}{2}$ |
| K4 | 0 | 0.7 | 0 | $-\frac{\pi}{2}$ |
| K5 | 0 | 0 | 0 | $\frac{\pi}{2}$ |
| K6 | 0 | 0.1735 | 0 | 0 |


## 3.3 Dynamics

This section lays out the theoretical groundwork for understanding the dynamics of serial manipulators. The derivation of  differential equations of motion for a generic open-chain robotic manipulator with $n$ degrees of freedom (dof) is done following the Euler-Lagrange Formulation. These differential equations form the dynamic model of the robot, describing the relationship between joint positions ($q$), velocities ($\dot{q}$), and accelerations ($\ddot{q}$) with the torques ($\tau$) applied to these joints by actuators.

Following this, the topic of friction, its modeling and algorithmic implementation is introduced. The final part of the section addresses the calculation and algorithmic implementation necessary for converting torques at the Tool Center Point (TCP) to joint torques and vice versa.

### 3.2.1 Euler-Lagrange Formulation


This is an energy-based approach to derive the dynamic equations in symbolic form of a rigid multibody system. This approach provides a systematic way, independently from the coordinate reference frame, to derive the equation of motion. In order to compute the Lagrangian of the robot manipulator, denoted by $L$, the potential energy $U_i$ and the kinetic energy $T_i$ must be computed for each link $i$ with $i = 1, ..., n$. The kinetic energy of a generic link i consists of three contributions:
a translational contribution due to the fact that the center of mass of the link has a non-zero absolute velocity $v_{c,i}$ , a rotational contribution due to the fact that the link is rotating with an absolute angular velocity $\omega_i$ and a mutual contribution which can be shown that when modeling within the center of gravity (cog) is zero, thus
$$
T_i =\frac{1}{2} m_i v^T_{c,i} v_{c,i} + \frac{1}{2}\omega^T_i I_{c,i}\omega_i 
$$

Where $m_i$ is the mass of link $i$ and $I_{c,i}$ represents the inertia tensor relative to the cog of link $i$. One can notice that to obtain the robot’s kinetic energy, it is necessary to compute the differential kinematics for each cog. Consequently, it involves the computation of n “partial” geometric Jacobians. For example, at link $i$,this geometric Jacobian is composed of the first $i$ columns, in general different from zero, and the last $n − i$ columns are zero i.e.,
$$
\begin{pmatrix}
    \mathbf{v}_{c,i} \\
    \boldsymbol{\omega}_i
\end{pmatrix} =
\begin{pmatrix}
    \mathbf{J}_{p,i}^{(1)} & \cdots & \mathbf{J}_{p,i}^{(i)} & 0 & \cdots & 0 \\
    \mathbf{J}_{o,i}^{(1)} & \cdots & \mathbf{J}_{o,i}^{(i)} & 0 & \cdots & 0
\end{pmatrix}
\dot{\mathbf{q}}
$$
Here, $j^{(1)}_{p,i}$ denotes the ith column of the first three lines of the geometric Jacobian, which relates the velocity in the joint space with the linear velocity in the task space.

Analogously,$j^{(1)}_{o,i}$denotes the ith column of the last three lines of the geometric Jacobian, which relates the velocity in the joint space with the angular velocity in the task space. The total kinetic energy T can be obtained by summing all the kinetic energy of each link which is given by
$$
T = \sum_{i=1}^{n} T_i
$$
The potential energy of a generic infinitely rigid stiff link $i$ depends only on the configuration of the robot and is given by
$$
U_i = -m_i \bar{\mathbf{g}}^\mathrm{T} \mathbf{r}_{0,ci}
$$
where ḡ is the gravity acceleration vector in the base frame and r 0,ci is the positionof the cog of link i expressed in reference frame RF 0 . The potential energy of the robot is given by
$$
U = \sum_{i=1}^{n} U_i
$$
The Lagrangian can be written as the difference between the kinetic energy and the potential energy as follows
$$
L = T - U
$$
The Lagrangian equation is expressed by
$$
\frac{d}{dt} \left( \frac{\partial \mathcal{L}}{\partial \dot{\mathbf{q}}} \right)^\mathrm{T} - \left( \frac{\partial \mathcal{L}}{\partial \mathbf{q}} \right)^\mathrm{T} = \mathbf{u},
$$
where $u$ are the generalized forces given by the nonconservative forces, which typically are: friction torques, torques provided by the actuators and torques induced by contact between the robot and the environment. Rearranging the equation yields the dynamical model in a compact form
$$
M(\mathbf{q}) \ddot{\mathbf{q}} + \mathbf{c}(\mathbf{q}, \dot{\mathbf{q}}) + \mathbf{g}(\mathbf{q}) = \mathbf{u}.
$$
Here, $M (q) ∈ R^{n×n}$ is a symmetric and positive definite matrix representing the robot’s inertia, $c(q, q̇) ∈ R^n$ is the vector containing the Coriolis, centrifugal and gyroscopic terms and $g(q) ∈ R^n$ is the gravity vector. The detailed derivation and more discussions can be found in [B. Siciliano, L.Sciavicco, L. Villani, and G. Oriolo. Robotics: Modelling, planning
and control. Springer, 2009.]

#### torque feed forward

### 3.2.2 Friction Model

Friction is a natural nonlinear phenomenon that is hard to model. Many simplifications can be done in order to have a satisfactory description of the friction phenomena. Various friction models have been proposed in the literature. A comprehensive overview, which discusses relevant analysis techniques and friction compensation methods, can be found in [21]. The static model provides a map between the joint velocity and friction force. The main friction phenomena that these models are able to capture are:
* Coulomb friction F c being a force generated when two moving parts have a
relative velocity v, characterized by
$$ 
F_c(F_n, v) = \mu F_n \operatorname{sgn}(v).
$$
Here, $F_n$ is the force proportional to the normal load, μ is the frictional factor, and sgn( )  ̇ is the sign function. Since the sign function is present, this model cannot handle the case where the relative velocity is zero. The friction force at zero may be any values in the range [−F c , F c ] but sgn(0) = 0.
* Viscous friction F v , in a robotic manipulator, is mainly caused by the viscosity
of the lubricant and is proportional to the sliding velocity [22], given by
$$ 
F_v(v) = \beta v,
$$
where β is the viscous friction coefficient.
* Static friction, also called stiction F s is not taken into account by the two previous models, therefore should be modeled separately. Since this effect is
present, higher friction forces are generated when the robot is at rest, and motion can start only when the applied forces exceed F s . Static friction is
characterized by
$$ 
F_s(F_b, F_\text{ext}) =
\begin{cases}
F_\text{ext} & \text{if } v = 0 \quad \|F_\text{ext}\| < F_b \\
F_b \operatorname{sgn}(F_\text{ext}) & \text{if } v = 0 \quad \|F_\text{ext}\| \geq F_b \\
\end{cases},
$$
In this thesis, the friction model proposed in [] is used which is a continuously differentiable model and captures all the 3 friction components mentioned above. 

### 3.2.3 Conversion of TCP forces to Joint torques


The conversion of external forces and torques applied to the end-effector of a 6-DOF robot to joint torques is achieved through the Jacobian matrix $\mathbf{J}(\mathbf{q})$, which relates joint velocities to the end-effector linear and angular velocities.

#### External Force and Torque Representation

External forces $\mathbf{F}_{\text{ext}}$ and torques $\mathbf{\tau}_{\text{ext}}$ are combined into a wrench vector $\mathbf{W}_{\text{ext}}$:

$$
\mathbf{W}_{\text{ext}} = \begin{bmatrix} \mathbf{F}_{\text{ext}} \\ \mathbf{\tau}_{\text{ext}} \end{bmatrix}
$$

#### Jacobian Matrix

The Jacobian matrix $\mathbf{J}(\mathbf{q})$ maps joint velocities to the end-effector velocities:

$$
\mathbf{J}(\mathbf{q}) = \begin{bmatrix} \mathbf{J}_v(\mathbf{q}) \\ \mathbf{J}_\omega(\mathbf{q}) \end{bmatrix}
$$

#### Conversion to Joint Torques

The joint torques $\mathbf{\tau}_{\text{joint}}$ are computed by:

$$
\mathbf{\tau}_{\text{joint}} = \mathbf{J}^\top(\mathbf{q}) \mathbf{W}_{\text{ext}}
$$

This equation projects the external wrench into the joint space, providing the necessary torques at each joint to counteract the external forces and torques.

In summary, the Jacobian matrix serves as the key transformation tool, allowing for the translation of external forces and torques applied at the robot's end-effector into corresponding joint torques, enabling effective control and response to external interactions.

Note that the Jacobain and the wrench are defined in the same coordinates.

In [2]:
#implementation here

## 3.4 Machine Learning
NN, RNN; GRU, LSTM, Pytorch, training loop.

## References

<!-- BIBLIOGRAPHY START -->
<div class="csl-bib-body">
  <div class="csl-entry"><i id="zotero|17740253/3J5DS9LQ"></i>Denavit, J., &#38; Hartenberg, R. S. (2021). A Kinematic Notation for Lower-Pair Mechanisms Based on Matrices. <i>Journal of Applied Mechanics</i>, <i>22</i>(2), 215–221. <a href="https://doi.org/10.1115/1.4011045">https://doi.org/10.1115/1.4011045</a></div>
  <div class="csl-entry"><i id="zotero|17740253/LY2FP5ZV"></i>Ghosal, A. (2006). <i>ROBOTICS: FUNDAMENTAL CONCEPTS AND ANALYSIS</i>. OUP India. <a href="https://books.google.de/books?id=GsRhAAAACAAJ">https://books.google.de/books?id=GsRhAAAACAAJ</a></div>
</div>
<!-- BIBLIOGRAPHY END -->